In [1]:
# imports 
import pandas as pd
import numpy as np
from Survival_prob_Class import survival_prob

In [2]:
# initialize simulation 
surv_prob = survival_prob()

In [3]:
# read the simulation datasets 
sim_05_1 = pd.read_csv('Data/simulation_half_1st.csv')
sim_05_2 = pd.read_csv('Data/simulation_half_2nd.csv')
sim_05_3 = pd.read_csv('Data/simulation_half_3rd.csv')
sim_1_1 = pd.read_csv('Data/simulation_one_1st.csv')
sim_1_2 = pd.read_csv('Data/simulation_one_2nd.csv')
sim_1_3 = pd.read_csv('Data/simulation_one_3rd.csv')

In [4]:
# concatenate the three simulation datasets per percentage 
# 0.5%
sim_05 = pd.concat([sim_05_1, sim_05_2, sim_05_3])
# reset index 
sim_05 = sim_05.reset_index(drop=True)
# 1% 
sim_1 = pd.concat([sim_1_1, sim_1_2, sim_1_3])
# reset index 
sim_1 = sim_1.reset_index(drop=True)

In [5]:
# remove unecessary columns, only need duration_Responder, duration_AED and duration_Vector 
# columns to drop
columns_to_drop = ['Patient_loc', 'Responder_lon', 'Responder_lat', 'AED_lon', 'AED_lat', 'Vector_lon', 'Vector_lat']  
sim_05.drop(columns=columns_to_drop, inplace=True)
sim_1.drop(columns=columns_to_drop, inplace=True)

In [6]:
# calculate time for the slowest vector 
np.max(sim_1['duration_Vector'])

703.8

In [7]:
# in some cases the AED never arrives (it is too far away), need to put this to a value for Survival_prob_Class.py
# choose a very large value (10 000) then we know that the vector always will be faster, slowest vector 703.8 s 

# find instances where duration_AED is 'No AED' and replace them with 10000
sim_05['duration_AED'] = sim_05['duration_AED'].replace('No AED', float(10000))
sim_1['duration_AED'] = sim_1['duration_AED'].replace('No AED', float(10000))


In [18]:
# in the case we don't have a responder, too far away from the patient
# put time of patient to something very large -> same probability of survival with and without fcr, what we would have with only vector

# find instances where duration_Respinder is 'No responder' and replace them with 10000
sim_05['duration_Responder'] = sim_05['duration_Responder'].replace('No responder', float(10000))
sim_1['duration_Responder'] = sim_1['duration_Responder'].replace('No responder', float(10000))

In [30]:
# estimate probability of survival

# save probabilites in df
db_prob_05 = pd.DataFrame(columns=['prob_resp', 'prob_vec'])
df_prob_1 = pd.DataFrame(columns=['prob_resp', 'prob_vec'])

# loop through df 0.5 
for index, row in sim_05.iterrows():
    prob_resp, prob_vec = surv_prob.probability_survival(float(row['duration_Responder']), float(row['duration_AED']), row['duration_Vector'])
    # add calcualted probabilities to df 
    db_prob_05.loc[index] = {'prob_resp': prob_resp, 'prob_vec': prob_vec}

# loop through df 1
for index, row in sim_1.iterrows():
    prob_resp, prob_vec = surv_prob.probability_survival(float(row['duration_Responder']), float(row['duration_AED']), row['duration_Vector'])
    # add calcualted probabilities to df 
    df_prob_1.loc[index] = {'prob_resp': prob_resp, 'prob_vec': prob_vec}